# Regresia liniara

**Termen de predare: 4 noiembrie 2022, ora 20:00**

Se vor folosi type annotations pentru variabile, parametrii tuturor funcțiilor, tipuri de retur. Se vor folosi docstrings pentru toate funcțiile. Neîndeplinirea acestei cerințe duce la înjumătățirea punctajului pentru exercițiul în cauză.

Se acordă doua puncte din oficiu. Fișierul va fi denumit tema4_ia_nume_prenume.ipynb. Verificați înainte de trimitere faptul ca execuția celulelor de sus în jos funcționează corespunzător. Aserțiunile sunt obligatoriu a fi indeplinite. Suplimentar, puteti face verificari si pentru alte valori.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from typing import Tuple, List

## Citirea datelor

In [ ]:
# data source: https://www.kaggle.com/datasets/quantbruce/real-estate-price-prediction

path = './data/Real estate.csv'
full_data = pd.read_csv(path, index_col='No')

# Primele 10 linii
full_data.head(10)

In [ ]:
# stergem coloana 'X1 transaction date'
full_data.drop(labels='X1 transaction date', axis=1, inplace=True)

## Selectare subseturi de antrenare si de testare

Setul de date initial se imparte in doua: pe 70% din date se face antrenarea modelului - determinarea ponderilor $\theta$, iar pe restul se face testare, pentru a vedea cat de bine generalizeaza modelul. 

Intrucat ordinea initiala a datelor ar putea defavoriza modelul (in datele de mai sus, primele inregistrari sunt cele mai vechi vanzari; cele mai recente sunt ultimele din fisier), decidem sa amestecam in prealabil datele si apoi sa facem impartirea lor. 

In [ ]:
# amestec aleator al inregistrarilor
full_data = full_data.sample(frac=1)

# split de date
percent_train = 0.7  # 70% din intregul set este folosit pentru antrenare
# restul pentru testare

# setez numarul de inregistrari care vor aparea in setul de antrenare; 
# restul se vor folosi in setul de testare
train_items_count = ...
# in train vom selecta primele 70% din date si transformam in numpy array
train = ...
test = ...

assert isinstance(train, np.ndarray), 'Train set should be an array'
assert isinstance(test, np.ndarray), 'Test set should be an array'

assert len(train) + len(test) == len(full_data)
assert train.shape[1] == test.shape[1] == len(full_data.columns), 'The number of columns should be the same'

In [ ]:
# ultima coloana din x_train, x_test sunt valoarile ce se prezic, restul sunt atributele de intrare

x_train, y_train = ...
x_test, y_test = ...

assert x_train.shape[0] == train.shape[0]
assert x_test.shape[0] == test.shape[0]

assert y_train.shape == (x_train.shape[0], 1), f'Vectorul y_train este vector coloana'
assert y_test.shape == (x_test.shape[0], 1), f'Vectorul y_test este vector coloana'

Functiile de mai jos sunt utilizate pentru transformarea convenabila a datelor:

Pentru scalarea valorilor din setul de instruire intre 0 si 1 e nevoie sa stim minimul si maximul pentru fiecare atribut. Pentru o matrice de valori de forma
$$
\mathbf{X} = 
\begin{pmatrix}
x_0^{(1)} & x_1^{(1)} & x_2^{(1)} & \dots & x_n^{(1)}
\\
x_0^{(2)} & x_1^{(2)} & x_2^{(2)} & \dots & x_n^{(2)}
\\
\vdots & \vdots & \vdots	& \vdots & \vdots
\\
x_0^{(1)} & x_1^{(m)} & x_2^{(m)} & \dots & x_n^{(m)}
\end{pmatrix}
$$
se obtin: minimele si maximele pentru fiecare coloana $min_i$, $max_i$, se scade din fiecare valoare minimul de pe coloana sa si se imparte la diferenta intre maximul si minimul pe acea coloana. Trebuie avuta in vedere situatia in care o coloana e constanta - de exemplu prima coloana 1, in matricea de design.

## Functii de preprocesare

In [ ]:
def get_min_max_cols(mat: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Pentru o matrice data, calculeaza si returneaza maximele si minimele pe fiecare coloana.
    
    :param mat: matrice cu cel putin doua linii
    :return: minimele si de maximele pe coloane. 
    """
    mins, maxes = ...
    # se recomanda ca sa se returneze minimele si maximele ca matrice cu cate o linie. 
    # Indicatie: folositi parametrul keepdims
    assert mins.shape == maxes.shape == (1, mat.shape[1])
    return mins, maxes

Matricea de design are pe prima coloana valoarea 1, pentru a permite termen liber. Restul coloanelor sunt din datele citite

In [ ]:
def get_design_matrix(mat: np.ndarray) -> np.ndarray:
    """
    Functia adauga o coloana de 1 la matricea :param mat:, pentru a permite un 
    termen liber in modelul liniar. Matricea data ca parametru nu se modifica.
    
    :param mat: matricea de la care se pleaca. contine doar valori de atribute
    :return: matrice de design, provenind din coloana plina cu 1 adaugata 
    inaintea matricei mat
    """
    ....
    result = ...
    assert result.shape == (mat.shape[0], mat.shape[1]+1)
    assert np.all(result[:, 0] == 1)
    return result

Implementam scalarea datelor la intervalul $[0, 1]$:

In [ ]:
def scale_values(mat: np.ndarray, min_cols: np.ndarray, max_cols: np.ndarray) -> np.ndarray:
    """
    Primind minimele si maximele pentru :param mat:, obtine matricea scalata: 
    din valorile de pe coloana i se scade minimul de indice i, adica min_cols[i] 
    si se imparte la (max_cols[i] - min_cols[i])
    
    :param mat: matricea pentru care se doreste obtinerea unei matrice scalate
    :param min_cols: minimele pe fiecare coloana
    :param max_cols: maximele pe fiecare coloana
    :return: o matrice de aceeasi forma ca si :param mat:, cu valorile scalate in intervalul [0, 1]
    """
    
    assert isinstance(mat, np.ndarray) and mat.ndim == 2, 'mat trebuie sa fie o matrice'
    assert mat.shape[0] > 1, 'cel putin doua coloane'
    assert mat.shape[1] == min_cols.shape[-1] == max_cols.shape[-1]
    
    # se trateaza cazul in care o coloana este constanta, i.e. minimul si maximul pe ea coincid. 
    # aceste coloane nu vor face impartirea, pentru ca ar rezulta o impartire la 0
    ...
    
    result = ...
    
    assert result.shape == mat.shape, 'Forma matricei se pastreaza'
    
    return result

In [ ]:
# testare cod

_mat = np.random.randn(1000, 1000) * 50
min_mat, max_mat = get_min_max_cols(_mat)
_mat_scaled = scale_values(_mat, min_mat, max_mat)

epsilon = 1e-5  # valoare pozitiva foarte mica; calculele in floating point pot duce la valori "aproximativ 0" sau 
# "aproximativ 1" 

assert np.min(_mat_scaled) >= -epsilon, f'Matricea nu e scalata in [0, 1]: minimul ei este {np.min(_mat_scaled)}'
assert np.max(_mat_scaled) <= 1 + epsilon, f'Matricea nu e scalata: maximul ei este {np.max(_mat_scaled)}'

_mat = get_design_matrix(_mat)  # adauga o coloana de 1
min_mat, max_mat = get_min_max_cols(_mat)
_mat_scaled = scale_values(_mat, min_mat, max_mat)
assert np.min(_mat_scaled) >= -epsilon, f'Matricea nu e scalata in [0, 1]: minimul ei este {np.min(_mat_scaled)}'
assert np.max(_mat_scaled) <= 1 + epsilon, f'Matricea nu e scalata: maximul ei este {np.max(_mat_scaled)}'


Se preproceseaza setul de antrenare: matrice de design -> scalare:

In [ ]:
# obtinem matricea de design
x_train = ...

# calculam minime si maxime pe coloane, din setul de antrenare
# acestea vor fi folosite si pentru transformarea datelor din setul de testare
x_train_min, x_train_max = ...

# scalam datele la intervalul [0, 1], folosim scale_values
x_train_scaled = ...

epsilon = 1e-5  # valoare pozitiva foarte mica

assert np.min(x_train_scaled) >= -epsilon, f'Matricea nu e scalata in [0, 1]: minimul ei este {np.min(_mat_scaled)}'
assert np.max(x_train_scaled) <= 1 + epsilon, f'Matricea nu e scalata: maximul ei este {np.max(_mat_scaled)}'

Pentru setul de testare se face acelasi tip de preprocesare, cu observatia ca pentru scalare se vor folosi valorile din `x_train_min` si `x_train_max`:

In [ ]:
x_test = get_design_matrix(x_test)

x_test_scaled = scale_values(x_test, x_train_min, x_train_max)

print(f'Ar trebui ca valorile minime din setul de testare pe fiecare coloana sa nu fie mult diferite fata de 0, exceptand prima coloana')
print(f'minim pe coloane pentru setul de testare: {np.min(x_test_scaled)}')

print(f'Ar trebui ca valorile maxime din setul de testare pe fiecare coloana sa nu fie mult diferite fata de 1. Pentru prima coloana trebuie sa fie chiar 1')
print(f'minim pe coloane pentru setul de testare: {np.max(x_test_scaled)}')

## Model de regresie liniara

Modelul de predictie este:
$$
h_\theta(X) = X \cdot \theta
$$
unde $X$ este matrice de design iar $\theta$ vector de ponderi. 

In [ ]:
def model_predict(theta: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Efectueaza predictii pentru valorile din matricea :param x:, folosind
    parametrii din :param theta:.
    
    :param theta: matrice coloana de ponderi
    :param x: matrice de design. Fiecare linie contine valorile atributelor 
    pentru o inregistrare.
    
    
    :return: matrice coloana de predictii, cu acelasi numar de linii ca si 
    :param x:.
    """
    # numarul de trasaturi din x este egal cu numarul de coeficienti din theta
    assert x.shape[1] == theta.shape[0]
    # vectorul theta e vector coloana
    assert theta.shape[1] == 1
    result = ...
    assert result.shape == (x.shape[0], 1)
    return result

In [ ]:
# verificari

np.random.seed(100)

_x = np.random.rand(3, 2)
_y = np.random.rand(3, 1)
_theta = np.ones((2, 1))

assert np.allclose(model_predict(_theta, _x), np.array([[0.82177433],
       [1.26929372],
       [0.12628798]]))

## Functia de eroare (de cost, loss function)

Functia de eroare masoara cat de diferite sunt predictiile fata de valorile cunoscute (ground truth). Aici folosim ca functie de eroare jumatate din [eroarea patratica medie](https://en.wikipedia.org/wiki/Mean_squared_error):

In [ ]:
def loss(theta: np.ndarray, x: np.ndarray, y: np.ndarray) -> float:
    """
    Functie de eroare folosita pentru calcul de gradienti. 
    
    :param theta: matrice coloana de ponderi
    :param x: matrice de design. Fiecare linie contine valorile atributelor 
    pentru o inregistrare.
    :param y: valorile de iesire asociate fiecarei inregistrari.
    
    :return: valoare scalara, reprezentand eroarea.
    """
    m = x.shape[0]
    y_hat = model_predict(theta, x)
    ...
    result = ...
    return result

In [ ]:
np.random.seed(100)

_x = np.random.rand(3, 2)
_y = np.random.rand(3, 1)
_theta = np.ones((2, 1))

assert np.allclose(loss(_theta, _x, _y), 0.03659284388808936)

## Calcul de gradienti si antrenare

In [ ]:
def gradient(theta: np.ndarray, x: np.ndarray, y: np.ndarray) -> np.ndarray:
    """
    Calculeaza gradientul pentru un model liniar cu parametrii :param theta:,
    pentru intrarea :param x: si iesirea :param y:
    
    :param theta: matrice coloana de parametri
    :param x: matricea de design cu valorile de intrare
    :param y: valorile de iesire
    :return: vectorul de valori de iesire 
    """
    
    assert theta.shape[0] == x.shape[1]
    m = x.shape[0]
    y_hat = model_predict(theta, x)
    assert y_hat.shape == (m, 1)
    ...
    grad = ...
    assert grad.shape == theta.shape
    return grad

In [ ]:
def train(x: np.ndarray, y: np.ndarray, alpha:float=0.1, max_iters = 10000, verbose=True) -> Tuple[np.ndarray, List[float]]:
    """
    Antreneaza modelul liniar pe setul de antrenare :param x:, :param y:. 
    Returneaza lista de valori de eroare si vectorul de ponderi la care s-a ajuns
    
    :param x: matricea de design cu valorile de intrare
    :param y: valorile de iesire
    :param alpha: rata de invatare, valoare implicita 0.1
    :param max_iters: numarul maxim de iteratii permise. Daca se ajunge la acest prag se opreste antrenarea. 
    :param verbose: daca e True, atunci se va afisa la fiecare 1000 de iteratii: numarul iteratiei si loss-ul curent
    :return: un tuplu cu ponderile rezultate si lista de valori ale functiei de eroare
    """
    eps = 1e-6
    assert x.shape[0] == y.shape[0]
    assert np.all(x[:, 0] == 1), 'Prima coloana trebuie sa fie 1'
    m, n = x.shape
    theta = .... # matrice coloana cu valori de 0
    assert theta.shape == (n, 1)
    assert np.all(theta==0)
    
    losses = []
    losses.append(loss(theta, x, y))
    iters = 0
    while True:
        iters += 1
        theta = theta - alpha * gradient(theta, x, y)
        current_loss = loss(theta, x, y)
        losses.append(current_loss)
        if (np.abs(losses[-1] - losses[-2]) < eps) or (iters > max_iters):
            break
        if np.isnan(current_loss) or np.isinf(current_loss):
            break
        if verbose and iters % 1000 == 0:
            print(f'Iteratia {iters}; loss = {current_loss}')
    return theta, losses

In [ ]:
theta, losses = train(x_train_scaled, y_train, alpha=0.5)

In [ ]:
print(f'losses={losses}')

In [ ]:
plt.plot(losses)
plt.xlabel('Epoca')
plt.ylabel('Eroare')
plt.show()

## Testarea modelului 

In [ ]:
# functie mean squared error

def mse(y_true: np.ndarray, y_hat: np.ndarray) -> float:
    """
    Calculeaza mean squared error pentru vectorii :param y_true: si :param y_hat:.
    
    :param y_true: vector de valori ground truth
    :param y_test: vector de valori prezisi de un model.
    
    :param return: mean squared error
    """
    
    return 1/y_true.shape[0] * np.sum((y_true - y_hat)**2)

### Testarea pe setul de antrenare

In [ ]:
y_hat_train = ...
    
print(f'Mean squared error pentru setul de antrenare: {mse(y_hat_train, y_train)}')

In [ ]:
print(f'Valori actuale si prezise, set de antrenare: {list(zip(y_train, y_hat_train))}')

### Testare pe setul de testare

In [ ]:
y_hat_test = ...
    
print(f'Mean squared error pentru setul de testare: {mse(y_hat_test, y_test)}')

In [ ]:
print(f'Valori actuale si prezise, set de testare: {list(zip(y_test, y_hat_test))}')

Cerinte:
1. Gasiti o valoare pentru alpha (learning rate) pentru care algoritmul produce o lista de valori crescatoare pentru functia de cost. Faceti o reprezentare grafica a valorilor din losses. Ce constatati legat de vectorul de ponderi `theta`?
1. Gasiti o valoare pentru alpha>0 pentru care algoritmul nu se opreste in `max_iters` iteratii. Faceti o reprezentare grafica a valorilor din losses.

In [ ]:
# Rezolvare cerinta 1
theta, losses = train(x_train_scaled, y_train, alpha=...)
theta  

In [ ]:
plt.plot(losses)
plt.show()

In [ ]:
losses  

In [ ]:
# Rezolvare cerinta 2
theta, losses = train(x_train_scaled, y_train, alpha=...)
losses 



In [ ]:
plt.plot(losses)
plt.show()

## Metoda ecuatiilor normale

Pentru regresia liniara se pot determina ponderile $\theta$ folosind metoda ecuatiilor normale. Formula pentru $\theta$ este:

$$
{\theta}^{(min)} = \left(\mathbf{X}^t \mathbf{X}\right)^{-1}\mathbf{X}^t\cdot \mathbf{y}
$$
unde $\mathbf{X}$ este matricea de design, $\mathbf{y}$ este vectorul de valori ground truth. 

Pentru determinarea pseudoinversei $\left(\mathbf{X}^t \mathbf{X}\right)^{-1}\mathbf{X}^t$ a matricei $\mathbf{X}$ se poate folosi functia `np.linalg.pinv` din [NumPy](https://numpy.org/doc/stable/reference/generated/numpy.linalg.pinv.html).

Determinati vectorul de valori de ponderi folosind metoda pseudoinversei si comparati cu vectorul de ponderi determinat prin metoda gradientului, pentru un $\alpha$ care duce la convergenta antrenarii.

Observatie: in metoda ecuatiilor normale nu este nevoie de a face scalarea/standardizarea datelor, spre deosebire de metoda gradient descent. Pentru comparatie, insa, folositi matricea scalata `x_train_scaled`.

In [ ]:
theta_normal = ...
print(f'Vectorul de ponderi determinat prin metoda ecuatiilor normale este:\n{theta_normal}')

In [ ]:
# determinam un vector de ponderi prin metoda gradient descent, folosind setul de antrenare
theta_gradient, _ = ...
print(f'Vectorul de ponderi determinat prin metoda gradient descent este:\n{theta_gradient}')

In [ ]:
print(f'Distanta intre vectorii determinati prin cele doua metode: {np.linalg.norm(theta_gradient - theta_normal)}')

## Selectare de atribute

In setul de date initial sunt prezente si latitudinea si longitudinea proprietatilor vandute - coloanele X5 si X6, adica ultmele doua din matricele x_train si x_test:

In [ ]:
full_data.head()

Ne punem intrebarea daca scoaterea lor imbunatateste performanta modelului de predictie. Implementati si executati urmatorii pasi:

1. Stergeti ultimele doua coloane din x_train si x_test;
1. Reapelati functiile de preprocesare pe matricele date; e nevoie de acest lucru, pentru ca vectorii de minime si maxime pe coloane vor avea acum cu 2 elemente mai putin.
1. Reantrenati modelul cu metoda gradient descent si calculati MSE pe setul de antrenare si de testare. Comparati cu valorile obtinute in sectiunea de testare, atat pentru setul de antrenare, cat mai ales pentru setul de testare. Cum comentati rezultatele obtinute?